In [5]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [6]:
# PyTorch不会隐式地调整输入的形状。因此，
# 我们在线性层前定义了展平层（flatten），来调整网络输入的形状
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights)

In [7]:
loss = nn.CrossEntropyLoss(reduction='none')

In [8]:
trainer = torch.optim.SGD(net.parameters(), lr=0.1)

In [9]:
def accuracy(y_hat,y):
    y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    # 不在此处除以batchsize是因为最后一个可能未读满导致错误
    return float(cmp.type(torch.float32).sum())

In [10]:
def sum_list(list):
    sum = 0
    for i in list:
        sum += i
    return sum

# 计算指定数据集上正确率
def data_accuracy(net,data_iter):
    acc = []
    total = 0
    with torch.no_grad():
        for X,y in data_iter:
            acc.append(accuracy(net(X),y))
            total += len(y)
    return sum_list(acc) / total
print(data_accuracy(net,test_iter))

0.0832


In [12]:
def train_module(net,data_iter,test_iter,loss,updater,epoch):
    for i in range(epoch):
        for X,y in data_iter:
            l = loss(net(X),y)
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        print(data_accuracy(net,test_iter))
train_module(net,train_iter,test_iter,loss,trainer,10)

0.7874
0.8139
0.8138
0.8207
0.8257
0.828
0.8299
0.8298
0.831
0.8294
